# NewsAPI Classification and Text Analysis
Authors: Christian Lee, Anahit Shekikyan, Graham Ward 

## Library Importing

## Data Importing

## Initial Descriptive Stats and Data Exploration

## Data Cleaning and Preprocessing

## Exploratory Data Analysis

## Data Splitting and Class Imbalance

## Model Development (Training, Validation and Testing)

## Topic Modeling

## Results